In [32]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

In [2]:
# 先定义消息函数和累和函数
gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

In [38]:
class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        if self.activation is not None:
            h = self.activation(h)
        return {'h' : h}

In [39]:
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(gcn_msg, gcn_reduce)
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [40]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gcn1 = GCN(1433, 16, F.relu)
        self.gcn2 = GCN(16, 7, None)
    
    def forward(self, g, features):
        x = self.gcn1(g, features)
        x = self.gcn2(g, x)
        return x

In [41]:
net = Net()
print(net)

Net(
  (gcn1): GCN(
    (apply_mod): NodeApplyModule(
      (linear): Linear(in_features=1433, out_features=16, bias=True)
    )
  )
  (gcn2): GCN(
    (apply_mod): NodeApplyModule(
      (linear): Linear(in_features=16, out_features=7, bias=True)
    )
  )
)


In [15]:
from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = data.graph
    g.remove_edges_from(nx.selfloop_edges(g))
    g = DGLGraph(g)
    g.add_edges(g.nodes(), g.nodes())
    return g, features, labels, train_mask, test_mask

In [16]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [36]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()

In [42]:
optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

/Users/Tree/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/Tree/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 2.0009 | Test Acc 0.1540 | Time(s) nan
Epoch 00001 | Loss 1.9728 | Test Acc 0.1880 | Time(s) nan
Epoch 00002 | Loss 1.9468 | Test Acc 0.2080 | Time(s) nan
Epoch 00003 | Loss 1.9202 | Test Acc 0.2070 | Time(s) 0.0165
Epoch 00004 | Loss 1.8928 | Test Acc 0.2200 | Time(s) 0.0159
Epoch 00005 | Loss 1.8649 | Test Acc 0.2410 | Time(s) 0.0156
Epoch 00006 | Loss 1.8365 | Test Acc 0.2570 | Time(s) 0.0156
Epoch 00007 | Loss 1.8073 | Test Acc 0.2780 | Time(s) 0.0156
Epoch 00008 | Loss 1.7774 | Test Acc 0.2910 | Time(s) 0.0159
Epoch 00009 | Loss 1.7473 | Test Acc 0.3100 | Time(s) 0.0161
Epoch 00010 | Loss 1.7178 | Test Acc 0.3270 | Time(s) 0.0160
Epoch 00011 | Loss 1.6887 | Test Acc 0.3560 | Time(s) 0.0159
Epoch 00012 | Loss 1.6601 | Test Acc 0.3910 | Time(s) 0.0158
Epoch 00013 | Loss 1.6322 | Test Acc 0.4370 | Time(s) 0.0158
Epoch 00014 | Loss 1.6048 | Test Acc 0.4630 | Time(s) 0.0157
Epoch 00015 | Loss 1.5781 | Test Acc 0.4810 | Time(s) 0.0156
Epoch 00016 | Loss 1.5523 | Test 